# Optimization Notebook
In this notebook, I handle hyperparameter tuning and performance evaluation.

In [ ]:
# Imports
import torch
from transformers import Trainer, TrainingArguments
from datasets import load_from_disk
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
# Load dataset
dataset = load_from_disk("./embeddings_imdb")

# Load fine-tuned model
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("./fine_tuned_bert")

In [ ]:
# Define metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# Define training arguments for optimization
training_args = TrainingArguments(
    output_dir="./optimized_results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=3e-5,  # adjusted learning rate for better performance
    weight_decay=0.01,
    logging_dir="./optimized_logs",
)

# Trainer with optimization settings
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics
)

# Fine-tune with optimized hyperparameters
trainer.train()

In [ ]:
# Evaluate model
results = trainer.evaluate()
print(results)